In [1]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import math
import sys
from scipy.stats import spearmanr

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
impIterative = IterativeImputer(missing_values=np.nan, sample_posterior=True, imputation_order='random')

from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [2]:
url = "0 ECLSK_98_99_K8_CHILD_v1_0.csv"

In [ ]:
original_dataset=read_csv(url, low_memory=False)

In [ ]:
original_dataset

## Determining the Best Y-Variable

In [ ]:
new_dataset = original_dataset.copy()
new_dataset = new_dataset.replace(" ", np.NaN)
new_dataset

### Remove the columns with the most NA values

In [ ]:
def remove_highest_empties(df):
    half_of_df = len(df) / 2
    column_names = []
    
    for column_name, _ in df.iteritems():
        if (df[column_name].isna().sum() > half_of_df):
            column_names.append(column_name)
    
    df = df.drop(columns=column_names)
    return df

In [ ]:
new_dataset = remove_highest_empties(new_dataset)
new_dataset

### Remove the NA values from the Y-Variable

##### Using 8th grade Math Scores as the Y-variable

In [ ]:
clean_df = new_dataset.copy().dropna(subset=["C6R4MPF"])
clean_df

### Remove the variable relating to 5th Grade and Above

In [ ]:
def remove_unusable_variables(df):
    unusable_columns = []
    
    for column_name, _ in df.iteritems():
        if ('5' in column_name) or ('6' in column_name) or ('7' in column_name) or ('8' in column_name):
            if (column_name != 'C6R4MPF'):
                unusable_columns.append(column_name)
    
    df = df.drop(columns=unusable_columns)
    return df

In [ ]:
clean_df = remove_unusable_variables(clean_df)
clean_df

### Determining the Correlation Between Values

In [ ]:
# def find_correlation(df):
#     return df.corr()
# # find the correlation between every column and the test column

In [ ]:
# corr_dict = find_correlation(clean_df)
# corr_dict

In [ ]:
# corr_dict['CS_TYPE2']

In [ ]:
# mask = (corr_dict['C6R4MPF'] > 0.3)
# mask

In [ ]:
# corr_dict['C6R4MPF'][mask]

In [ ]:
# corr_dict['C6R4MPF'].loc[((corr_dict['C6R4MPF'] > 0.3) | (corr_dict['C6R4MPF'] < -0.3))] # || < -0.8

In [ ]:
# mask.index.values

In [ ]:
# def find_covariance(df):
#     return df.cov()

In [ ]:
# cov_df = find_covariance(clean_df)
# cov_df

## Creating a new dataset with the columns we're using

In [ ]:
# dataset = original_dataset[["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP","C6R4MPF"]]

In [ ]:
# dataset = clean_df.copy()

In [ ]:
# dataset.shape

In [ ]:
# Number of Non-NA values in each column
# dataset.count()

In [ ]:
# Number of NA values in each column
# dataset.isna().sum()

In [ ]:
# dataset.groupby('C6R4MPF').size()

In [ ]:
# cleandf=dataset.copy().dropna(subset=["C6R4MPF"])

In [ ]:
# cleandf.shape

In [ ]:
# cleandf.isna().sum()

In [ ]:
# print(cleandf.groupby('C6R4MPF').size())

In [ ]:
# dataset= cleandf[cleandf["C6R4MPF"]>"4"]
# print(dataset.groupby('C6R4MPF').size())

In [ ]:
# dataset.shape

In [ ]:
# Group some categories together
# conditions = [
#     (dataset['C6R4MPF'] >= "5.0") & (dataset['C6R4MPF'] < "7.0"),
#     (dataset['C6R4MPF'] >= "7.0")& (dataset['C6R4MPF'] <= "9.0")]
# choices = ['C - Low', 'A - High']
# scores = np.select(conditions, choices)
# dataset.loc[:, ('Score')] = scores
# dataset

In [ ]:
#remove the C6R4MPF column, now that we have the 'Score' category 
# dataset=dataset.drop('C6R4MPF',1)
# dataset.head()

In [ ]:
# print(dataset.groupby('Score').size())

In [ ]:
# dataset2=dataset[dataset['Score']=='A - High'].copy()

In [ ]:
# dataset2=dataset2.append(dataset[dataset['Score']=='C - Low'].sample(n=4777))

In [ ]:
# print(dataset2.groupby('Score').size())

In [ ]:
# print(dataset2.groupby('Score').size())

In [ ]:
# sample_size = math.floor(0.75*len(dataset2))
# sample_size

In [ ]:
# Creating our sample of random lines
# traindf=dataset2.sample(n=sample_size)
# #Listing what rows our sample is using
# traindf.index

In [ ]:
# Printing training dataset which imputation will be performed on
# traindf

In [ ]:
# Defining our validation data as rows in dataset2 that are not in our training data
# validationdf=dataset2.drop(traindf.index)
# len(validationdf)

In [ ]:
# taking out all NA's from validation dataset
# validationdf=validationdf.dropna()
# len(validationdf)

In [ ]:
# Do imputation on ALL columns of TRAIN 
# X = traindf[["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP"]]

In [ ]:
# X

In [ ]:
# data_imputed=pd.DataFrame(impIterative.fit_transform(X),columns=["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP"])

In [ ]:
# data_imputed

In [ ]:
# len(data_imputed)

In [ ]:
#Create a new column in data_imputed that doesn't exist yet to tack on the y-variable
# data_imputed["Score"]=traindf['Score'].values

In [ ]:
# data_imputed

In [ ]:
# data_imputed.shape

In [ ]:
# print(data_imputed.groupby('Score').size())

In [ ]:
# Split-out train dataset
# array = traindf.values
# X_train = array[:,0:14]
# Y_train= array[:,14]
# X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
# Spot Check Algorithms
# models = []
# models.append(('LR', LogisticRegression(max_iter=10000, multi_class='multinomial')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', weights='distance')))
# models.append(('CART', DecisionTreeClassifier()))
# models.append(('NB', GaussianNB()))
# models.append(('SVM', SVC(gamma='scale')))
# models.append(('RF', RandomForestClassifier(criterion='gini', n_estimators=10000, n_jobs=2)))

In [ ]:
# # evaluate each model in turn
# results = []
# names = []
# for name, model in models:
# 	kfold = StratifiedKFold(n_splits=7, random_state=1, shuffle=True)
# 	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
# 	results.append(cv_results)
# 	names.append(name)
# 	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
# #Set validation set
# X_validation=validationdf.iloc[:,0:14]
# Y_validation=validationdf.iloc[:,14]

In [ ]:
# # Make predictions on validation dataset with SVM
 
# model = SVC(gamma='scale')
# model.fit(X_train, Y_train)
# predictions = model.predict(X_validation)

In [ ]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions))
# plot_confusion_matrix(model.fit(X_train, Y_train),X_validation,Y_validation,cmap=plt.cm.Blues)
# print(classification_report(Y_validation, predictions))

In [ ]:
# # Make predictions on validation dataset with RF
 
# model2 = RandomForestClassifier(criterion='gini', n_estimators=10000, n_jobs=2)
# model2.fit(X_train, Y_train)
# predictions2 = model2.predict(X_validation)

In [ ]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions2))
# plot_confusion_matrix(model2.fit(X_train, Y_train),X_validation,Y_validation,cmap=plt.cm.Blues)
# print(classification_report(Y_validation, predictions2))

In [ ]:
# feature_importance = model2.feature_importances_

# sorted_idx = np.argsort(feature_importance)[::-1]
# print("Feature Importance:\n")
# for i in range(len(X.columns)):
#     idx = sorted_idx[i]
#     print(f'{X.columns[idx]:20} {feature_importance[idx]:.3f}')